# Data Visualization Developer Workshop

In this data visualization workshop, we will be analyzing a real-world time series dataset to predict the future. This notebook demonstrates how to download and prepare a live dataset, create data visualizations, and train and deploy a DeepAR model on the AWS SageMaker platform.

### Traffic Accident Prediction
In our hypothetical scenario, we are trying to predict the risk of traffic accidents in the City of Milwaukee. 

## Python Setup

Import the modules we will be using in our notebook

In [ ]:
import datetime
import os
import sys
import time
import random
import json

from urllib.request import urlretrieve
import numpy as np
import pandas as pd

import boto3
import s3fs
import sagemaker
import matplotlib.pyplot as plt

In [ ]:
%run -i helper.py

## Dataset Download and Preparation

We will first download our [datasource](https://data.milwaukee.gov/dataset/trafficaccident) from the [City of Milwaukee Data Portal](https://data.milwaukee.gov/).

In [ ]:
url = "https://data.milwaukee.gov/dataset/5fafe01d-dc55-4a41-8760-8ae52f7855f1/resource/8fffaa3a-b500-4561-8898-78a424bdacee/download/trafficaccident.csv"
filename = "trafficaccident.csv"

In [ ]:
if not os.path.isfile(filename):
    print("downloading dataset, can take a few minutes depending on your connection")
    urlretrieve(url, filename, reporthook=progress_report_hook)
else:
    print("File found skipping download")

Then, we load and parse the dataset and convert it to a collection of Pandas time series, which makes common time series operations such as indexing by time periods or resampling much easier.

In [ ]:
csv_df = pd.read_csv(filename, sep=",", parse_dates=True, error_bad_lines=False)
#csv_df

In [ ]:
ts_series = pd.to_datetime(csv_df['CASEDATE'], format='%Y-%m-%d %H:%M:%S.%f')
ts_series = ts_series.sort_values()
ts_series = ts_series.dropna()
ts_series = ts_series.rename("timestamp")
#ts_series

Since accidents are recorded individually in the original dataset, we need to aggregate the count of accidents into 1 hour bins. Also, for this particular dataset, data prior to 2008 is incomplete. We will only keeps records starting at 1/1/2018.

In [ ]:
df = pd.DataFrame(index=ts_series.rename("timestamp"))
df['count'] = 1
df = df.resample('1H').sum()
df = df['2008':]
#df

## Data Visualization


For the following visualizations, we want with full years, 2008 - 2018 in our case. We will create a dataframe for our target year.

In [ ]:
df_2008_2018 = df['???':'???']
#df_2008_2018

### Graph Accidents Over Time

We will start with two simple line graphs. First graph will display the number of accidents over the entire set of data, allowing us to see trends spanning multiple years. Next the graph will overlay each year on one another, showing how years compare to one another as well as showing seasonality trends.

#### Steps


1. Resampling the dataframe will be necessary to show meaningful results. Experiment with no resampling and by the hour, week, month, etc. 

1. Next create a pivot table dataframe, with the month as the index and the years the columns. Aggregate using the sum function. 

1. Plot the resampled dataframe.

1. Plot the pivot dataframe.

1. Use the plt.show() to output graph

#### Notes
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

Frequencies used in reample function:

    Alias   Description
    D       calendar day frequency
    W       weekly frequency
    M       month end frequency
    MS      month start frequency
    Q       quarter end frequency
    QS      quarter start frequency
    A       year end frequency
    AS      year start frequency
    H       hourly frequency

In [ ]:
#Step 1
df_resample = df_2008_2018.resample('??').sum()

#Step 2
df_pivot = pd.pivot_table(
    df_2008_2018,
    index = df_2008_2018.index.month,
    columns = df_2008_2018.index.year,
    aggfunc = 'sum'
)

df_pivot

In [ ]:
#Step 3
# Plot df_resample using the plot() function 

#Step 4
# TODO: plot df_pivot using the plot() function


#Moving the legend to the upper right
plt.legend(loc='upper right')


#Step 5
plt.show()

### Histograms 
A histogram groups numbers into ranges, generally represented as a bar chart. A histogram is a plot that lets you discover, and show, the underlying frequency distribution (shape) of a set of continuous data.  


#### Steps
1. First create dataframes of your sample data grouped by hour or day, day of week, and month of year
2. Plot the grouped by dataframe, with a kind of 'bar'

#### Notes
Monday = 0  
Midnight - 1:00AM = 0  

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html  
https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties  
https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html  
https://matplotlib.org/tutorials/colors/colormaps.html 

In [ ]:
#Step 1
df_groupedby_hour = df_2008_2018.groupby(df_2008_2018.index.hour).sum();
df_groupedby_weekday = # TODO: try the same, but with weekday
df_groupedby_month = # TODO: try the same, but with month

In [ ]:
#Step 2
df_groupedby_hour.plot(kind='bar')
# TODO: plot the weekday
# TODO: plot the month
plt.show()

Add some labels to make the graphs easier to understand. There are a couple of different way to accomplish this. For our example, we will rename index and columns of our grouped data.

In [ ]:
df_groupedby_hour.rename(columns={'count':'Accidents'}, inplace=True)
df_groupedby_hour.index.name = #TODO: Choose an x-axis title

df_groupedby_weekday.rename(columns={'count':'Accidents'}, inplace=True)
df_groupedby_weekday.index.name = #TODO: Choose an x-axis title

df_groupedby_month.rename(columns={'count':'Accidents'}, inplace=True)                          
df_groupedby_month.index.name = #TODO: Choose an x-axis title

#TODO: plot the graphs again (hint: you can copy and past from last code block)

plt.show()

Finally, lets try to make the charts more compact. We will do this by creating subplots. Subplot are created on a grid system. We will use the plt.subplot() which creates a single subplot within a grid. this command takes three integer arguments: the number of rows, the number of columns, and the index of the plot to be created in this scheme, which runs from the upper left to the bottom right.

Our subplot grid will have a height of 1 and width of 3.

    add_subplot(1,3 x)
   
    +---+---+---+
    | 1 | 2 | 3 | <== x
    +---+---+---+


In [ ]:


fig = plt.figure(1, figsize=(15, 4))

# Divide the figure into a 1x3 grid, and give me the first section
ax1 = fig.add_subplot(1,3,1)

# Divide the figure into a 1x3 grid, and give me the second section
ax2 = fig.add_subplot(1,3,2)

# Divide the figure into a 1x3 grid, and give me the third section
ax3 = fig.add_subplot(1,3,3)

df_groupedby_hour.plot(kind='bar', ax=ax3)
df_groupedby_weekday.plot(kind='bar',ax=ax2)
df_groupedby_month.plot(kind='bar',ax=ax1)

#TODO: Try to rearange the order of th graphs

plt.suptitle('2008-2018 Accidents')
plt.show()

### Heat Map Graph
Let's create a heat map, comparing the accidents that occur on each day of the week by hour. A heat map displays the values of matrix in a two dimensions.

#### Steps
1. First create a dataframe of your sample data.
2. Next create a pivot table dataframe, with the day of the week as the index (y axis) and hour of the day as the column (x axis). Aggregate using the mean function. 
3. Plot the pivot dataframe as a heat map using the `imshow()` function. 

#### Notes
Monday = 0  
Midnight - 1:00AM = 0  

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html  
https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html  
https://matplotlib.org/tutorials/colors/colormaps.html  

In [ ]:
#Step 1
df_heatmap = #TODO: use the df[ ?? : ?? ] syntax to select just 2018

#Step 2
pv_heatmap = pd.pivot_table(
    df_heatmap, 
    index = df_heatmap.index.weekday,
    columns = df_heatmap.index.hour,
    aggfunc='mean')

plt.figure(figsize=(15,5))

#Step 3
im = plt.imshow(pv_heatmap,cmap='jet')

#TODO: Try to change the color to 'plasma' or another color from this link: https://matplotlib.org/tutorials/colors/colormaps.html
#TODO: adjust the size of the heat map by change the parameters of the plt.figure(...) function

plt.show()

## Machine Learning 

### SageMaker Setup

In [ ]:
sagemaker_session = sagemaker.Session()

Before starting, we will configure the values to configure the S3 buckets.

The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. 

The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.

In [ ]:
s3_bucket = sagemaker.Session().default_bucket()  # replace with an existing bucket if needed
s3_prefix = 'deepar-traffic-accident-notebook'    # prefix used for all data stored within the bucket
s3_uid = #TODO: Set a unique id                   # unique ID used for S3 data storage location

role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

In [ ]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/uid_{}/{}/data".format(s3_bucket, s3_uid, s3_prefix)
s3_output_path = "s3://{}/uid_{}/{}/output".format(s3_bucket, s3_uid, s3_prefix)
# print(s3_data_path)
# print(s3_data_path)

### Setup Test Data

To configure DeepAR, we need to split the data into training and testing data. For the training data, we will use data from 2008 through 2017, split by year. The training data will be all of 2018.

In [ ]:
time_series_training = []
time_series_training.append(df["2008":"2008"]['count'])
time_series_training.append(df["2009":"2009"]['count'])
time_series_training.append(df["2010":"2010"]['count'])
time_series_training.append(df["2011":"2011"]['count'])
time_series_training.append(df["2012":"2012"]['count'])
time_series_training.append(df["2013":"2013"]['count'])
time_series_training.append(df["2014":"2014"]['count'])
time_series_training.append(df["2015":"2015"]['count'])
time_series_training.append(df["2016":"2016"]['count'])
time_series_training.append(df["2017":"2017"]['count'])

time_series_test = []
time_series_test.append(df['2018':'2018']['count'])

### Write training and test data to S3

Now that we have the data arrays prepared, let us copy them to S3 where DeepAR can access them. This may take a couple of minutes, depending on your connection.

In [ ]:
encoding = "utf-8"
s3filesystem = s3fs.S3FileSystem()

with s3filesystem.open(s3_data_path + "/train/train.json", 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))

with s3filesystem.open(s3_data_path + "/test/test.json", 'wb') as fp:
    for ts in time_series_test:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))
        
#TODO - Navigate to S3 using the following link

https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-963348920600/?region=us-east-1&tab=overview

### Configure DeepAR estimator training job

We can now define the estimator that will launch the training job and configure the container image to be used for the region that we are running in.
#### Notes
https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.Estimator

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    base_job_name='DEMO-deepar',
    output_path=s3_output_path
)

### Set Hyperparameters
algorithm
Hyperparameters are parameters used for to configure machine learning algorithm, set are outside prior to the training process. Changing these parameters can have a dramatic impact on the accuracy and performance of the model. That said, tuning most of these hyperparameters is outside the scope of this workshop. The relevant hyperparameters are as follows:

 * **time_freq** - The target frequency interval of the data. Hourly in this example.
 * **context_length** - How far in the past DeepAR's network can see. Higher values may result in more accuracy at the expense of training time and expense, and computing resources to execute model.
 * **prediction_length** - How far in the future DeepAR's network can predict. Like **context_length**, higher values are more taxing during model training and execution. 

#### Notes
https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html  

In [ ]:
time_freq = 'H'
prediction_length = 336 # 2 weeks (24 * 14)
context_length = 336

In [ ]:
hyperparameters = {
    "time_freq": time_freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "55",
    "num_layers": "2",
    "likelihood": "gaussian",
    "epochs": "40",
    "mini_batch_size": "33",
    "learning_rate": "0.00256",
    "dropout_rate": "0.045",
    "early_stopping_patience": "10"
}
estimator.set_hyperparameters(**hyperparameters)

### Run DeepAR estimator job

We are ready to launch the training job. SageMaker will start an EC2 instance, download the data from S3, start training the model and save the trained model.

If you provide the test data channel, as we do in this example, DeepAR will also calculate accuracy metrics for the trained model on this test data set. This is done by predicting the last prediction_length points of each time series in the test set and comparing this to the actual value of the time series. The computed error metrics will be included as part of the log output.

Note: the next cell may take a few minutes to complete, depending on data size, model complexity, and training options.

<span style="color:red">**For the Data Visualization Developer Workshop, we have pre-trained models in the interest of time.**</span> 

In [ ]:
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

#estimator.fit(inputs=data_channels)

### Start Trained DeepAR Model Endpoint
Now that we have trained a model, we can use it to perform predictions by deploying it to an endpoint.

**Note:** Remember to delete the endpoint after running this experiment. A cell at the very bottom of this notebook will do that: make sure you run it at the end.

<span style="color:red">**For the Data Visualization Developer Workshop, we have pre-created an endpoint in the interest of time.**</span> 

In [ ]:
#job_name = estimator.latest_training_job.name
#
#endpoint_name = sagemaker_session.endpoint_from_job(
#    job_name=job_name,
#    initial_instance_count=1,
#    instance_type='ml.m4.xlarge',
#    deployment_image=image_name,
#    role=role
#)

endpoint_name = "DataWorkshopEndpoint"

Create an instance of the DeepARPredictor utility class. This will be used to help be used to make a request to the endpoint

In [ ]:
predictor = DeepARPredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type="application/json"
)
predictor.set_prediction_parameters(time_freq, prediction_length)

In [ ]:
time_series_predict = [time_series_training[9]]

Now we can use the previously created predictor object to generate the predicted data.

In [ ]:
df_prediction = predictor.predict(time_series_predict, quantiles=["0.1", "0.5", "0.9"])
df_prediction

### Plot Prediction 

Now that we have our prediction data, we will plot it against the test data.

#### Steps


1. Select the the resample interval, let we did early. Try between 1 and 6 hours. Also create a slice for the date range to plot. 2018-01-01 through 2018-01-07 is a good start. 

1. Create a dataframe from the prediction data based on the resample interval and date slice.

1. Get the series for the .1, .5, and .9 quartiles.

1. Setup plot area.

1. Plot the median line (.5 quartile)

1. Plot fill between the the .1 and .9 quartiles. This will give a shaded area that represents an 80% confidence interval.

1. Create a dataframe from the actual test data, based on the resample interval and date slice.

1. Plot the test data series

1. Use `plt.legend()` to add and legend and `plt.show()` to output graph

#### Notes
https://matplotlib.org/api/colors_api.html  

In [ ]:
# Step 1 - parameters for time resolution and range to play
plot_interval = '2h' #TODO: Try other intervals
plot_slice = slice('2018-01-01','2018-01-07') #TODO: Try 14 days

# Step 2 - Resampled and sliced prediction data
toplot_dataframe = df_prediction[0][plot_slice].resample(plot_interval).sum()

# Step 3 - Select quartiles to plot
toplotmedian = toplot_dataframe['0.5']
toplot10 = toplot_dataframe['0.1']
toplot90 = toplot_dataframe['0.9']

#Step 4 - Setup plot area. Let's make this one large.
plt.figure(figsize=(20,7))

#Step 5 - Plot median line and 
toplotmedian.plot(label='prediction median')

#Step 6 - Plot confidence interval area
plt.fill_between(toplot10.index, # x-axis
                 toplot10,       # bottom y-axis
                 toplot90,       # top y-axis
                 color='y',      
                 alpha=.5,       
                 label='80% confidence interval'
)

#Step 7 - Create test data series
actual_data_series = #TODO: resample and slice the time_series_test dataframe. See Step 2 for reference.

#Step 8 - Plot actual data series
#TODO: Plot the actual_data_series with a label of your choosing. See Step 5 for reference.

#Step 9 - Add a legend and show the graph
#TODO: Add a legend 
#TODO: Show the plot

### Delete endpoint

<span style="color:red">**Please do not delete the pre-created an endpoint!**</span> 

In [ ]:
#sagemaker_session.delete_endpoint(...)